# 2. Physics Informed Neural Networks

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
device = torch.device('cuda:0')

# Project - Finding Green's functions


We will solve the inhomogeneous Helmholtz equation :
$    \begin{equation}
        (\nabla^2 + k^2)u = u_{xx} + u_{yy} + u_{zz} + k^2 u = f \text{ for } {\rm {\bf r}} \in S
    \end{equation}$
for a given boundary $\partial S$ with the boundary conditions
$    \begin{equation}
        u({\rm {\bf r}}) = 0, \text{ for } {\rm {\bf r}} \in \partial S.
    \end{equation}$
To solve the equation, we can apply the Green's function $G$, which is a solution of the below equation:
$    \begin{equation}
        (\nabla^2 + k^2)G({\rm {\bf r}}, {\rm {\bf R}}) = \delta({\rm {\bf r - R}}).
    \end{equation}$
Thus the solution of the inhomogeneous equation is
$    \begin{equation}
        u({\rm {\bf r}}) = \int_S G({\rm {\bf r}}, {\rm {\bf R}})f({\rm {\bf R}}) d {\rm {\bf R}}.
    \end{equation}$

# Deep ritz method


    The loss function is given by
$    \begin{equation}
        \int_{S} \left( \frac{1}{2} |\nabla G_{nn}|^2 + \frac{1}{2} k^2 G_{nn}^2 - G_{nn}\delta({\rm{\bf r - R}}) \right) d{\rm \bf r} + \beta \cdot \int_{\partial S} G_{nn}(x,y)^2 d{\rm{ \bf r}}.
    \end{equation}$

$    \begin{equation}
        \int_{S} \left( \frac{1}{2} |\nabla G_{nn}|^2 + \frac{1}{2} k^2 G_{nn}^2 \right) d{\rm \bf r} - G_{nn}({\rm \bf R}, {\rm \bf R}) + \beta \cdot \int_{\partial S} G_{nn}(x,y)^2 d{\rm{ \bf r}}.
    \end{equation}$

Note that Lagrangian: https://en.wikipedia.org/wiki/Lagrangian_(field_theory)

# Relaxation

Since the $G_{nn} (R,R)$ is not a finite number in many cases, we should approximate the delta function with substitute functions(e.g. Gaussian).
By constructing a sequence of substitute functions that converge to the delta function, we can expect the solution could converge to Green's function.


# Mollifier 1: Gaussian

For example, using Gaussians,
$    \begin{equation}
        (\nabla^2 + k^2)G_{n}({\rm {\bf r}}, {\rm {\bf R}}) = M_n({\rm \bf r, R}) := \frac{1}{\sqrt{2\pi}\sigma}e^{-\frac{({\rm \bf r-R})^2}{n^2 \sigma^2}}
    \end{equation}$
where $\sigma$ is given. If $n \rightarrow \infty$, $G_n \rightarrow G$.


In this case, our loss function for each $n$ is
    $\begin{equation}
        \int_{S} \left( \frac{1}{2} |\nabla G_{nn}|^2 + \frac{1}{2} k^2 G_{nn}^2 - G_{nn}\frac{1}{\sqrt{2\pi}\sigma}e^{-\frac{({\rm \bf r-R})^2}{n^2 \sigma^2}} \right) d{\rm \bf r} + \beta \cdot \int_{\partial S} G_{nn}(x,y)^2 d{\rm{ \bf r}}.
    \end{equation}$


# Mollifier 2: bump function

$\begin{equation}
    M_n({\rm \bf r, R}) = \frac{1}{I_n}e^{-\frac{1}{1-|{\rm \bf r} - {\rm \bf R}|^2/\sigma^2}}
\end{equation}$

if $|{\rm \bf r} - {\rm \bf R}| < \sigma$. Otherwise, $M_n = 0$. $I_n$ is a normalization factor.